In [1]:
import tweepy
import pandas as pd
from queue import Queue
q = Queue()



In [2]:
# API Authentication

CONSUMER_KEY = 'hheqJPV07LZzIBCVGFEUt2ojI'
CONSUMER_SECRET = 'X3x4ausVn0KYLmclg6pAnPp4esnJ532J6z2Sla3QB1yAr37VKl'
ACCESS_TOKEN = '1065450120433553410-6H5N0JJYAdji06KNE3ozlgJj1zgeOL'
ACCESS_TOKEN_SECRET = '5kCiBBcDGOfwkmORFHD4cVVaNOAujUu29ML087KXsLlTn'

In [3]:


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


In [4]:
# Output details
out_user = '/Users/merrillsequeira/Documents/docs/UPGRAD_PGDDS_MODULES/PRACTICE_J/twitter_sentiment3/user_details.csv'
out_followers = '/Users/merrillsequeira/Documents/docs/UPGRAD_PGDDS_MODULES/PRACTICE_J/twitter_sentiment3/followers.csv'
out_skip = '/Users/merrillsequeira/Documents/docs/UPGRAD_PGDDS_MODULES/PRACTICE_J/twitter_sentiment3/skip.csv'




In [5]:

# Setting up data frames with initial user
user = api.get_user('@TheAcademy')
user_details = {'name':user.name,
     'screen_name':user.screen_name,
     'created_at':str(user.created_at),
     'id':user.id,
     'friends_count':user.friends_count,
     'followers_count':user.followers_count}
master_followers = pd.DataFrame({'id':user.id,'followers':user.followers_ids()})
master_user_details = pd.DataFrame([user_details])




In [6]:

# Setting up skip_listlist
skip_list = []
skip_list.append(user_details['id'])
skip_df = pd.DataFrame({'id':user.id},index=[0])


In [7]:
# Exporting initial master files
master_user_details.to_csv(out_user,index=False)
master_followers.to_csv(out_followers,index=False)
skip_df.to_csv(out_skip,index=False)

In [8]:
# Putting initial follower seed in queue
list(map(q.put,master_followers['followers']))
print (len(list(q.queue)))

5000


In [ ]:
while not q.empty():
    u = q.get()
    if u in skip_list:
        continue
    else:
            try:
                # API call to get user data
                user = api.get_user(u)
                user_details = {'name':user.name,
                                'screen_name':user.screen_name,
                                'created_at':str(user.created_at),
                                'id':user.id,
                                'friends_count':user.friends_count,
                                'followers_count':user.followers_count}
                
                # Adding to skip list
                skip_list.append(user_details['id'])
                skip_df = pd.DataFrame({'id':user.id},index=[0])
                
                # Appending user data to master list
                user_details = pd.DataFrame([user_details])
                master_user_details = master_user_details.append(user_details)
                
                # Getting followers and appending to master list
                followers = pd.DataFrame({'id':user.id,'followers':user.followers_ids()})
                if followers.shape[0] > 200:
                    followers = followers.sample(200)
                else:
                    pass
                master_followers = master_followers.append(followers)
                
                # Adding retrieved followers to queue
                list(map(q.put,followers['followers']))
                            
                # Exporting user and followers to CSV
                user_details.to_csv(out_user,index=False,mode='a',header=False)
                followers.to_csv(out_followers,index=False,mode='a',header=False)
                skip_df.to_csv(out_skip,index=False,mode='a',header=False)
                
                print (len(list(q.queue)))
                
            # Error handling
            except tweepy.TweepError as error:
                print (type(error))
        
                if str(error) == 'Not authorized.':
                    print ('Can''t access user data - not authorized.')
                    skip_list.append(u)
                    skip_df = pd.DataFrame({'id':u},index=[0])
                    skip_df.to_csv(out_skip,index=False,mode='a',header=False)
        
                if str(error) == 'User has been suspended.':
                    print ('User suspended.')
                    skip_list.append(u)
                    skip_df = pd.DataFrame({'id':u},index=[0])
                    skip_df.to_csv(out_skip,index=False,mode='a',header=False)    

5019
5218
5217
5233
5232
5234
5433
5492
5491
5577
5578
5609
<class 'tweepy.error.TweepError'>
Cant access user data - not authorized.


Rate limit reached. Sleeping for: 573
